In [28]:
import os
import compas

from compas.datastructures  import Mesh
from compas_view2.app import App

'''
# folder location
dirname = '/content/drive/My Drive/Colab Notebooks'


# 1. load blocks from Tutorial 7
#    Assignment : Load the blocks from your shell
blocks_in_name = '07_blocks_flat_top.json'
blocks_in_path = os.path.join(dirname, blocks_in_name)

blocks = [block for block in compas.json_load(blocks_in_path)]
'''

# TEMPORARY LOAD
blocks = [block for block in compas.json_load("07_blocks_flat_top_wirecutting.json")]

# 2. visualise the blocks
viewer = App(width=1600, height=900)
viewer.view.camera.ty = -0.8
viewer.view.camera.tx = -0.7
viewer.view.camera.distance = 10

for block in blocks:
    viewer.add(block)
viewer.show()

In [29]:
import os
import compas

from compas.datastructures  import Mesh
from compas_view2.app import App

my_block = blocks[0]

# 1. visualise individual block
viewer = App(width=1600, height=900)
viewer.view.camera.ty = -0.8
viewer.view.camera.tx = -0.7
viewer.view.camera.distance = 10

viewer.add(my_block)
viewer.show()

In [30]:
import os
import compas

from compas.datastructures  import Mesh
from compas.geometry  import Vector
from compas_view2.app import App

# print(faces_attribute(my_block,keys=top))
# print(my_block.faces_where({'top': True}))[0]

faces = list(my_block.faces())
print(faces)

'''################ EXPLAIN THIS ################'''

# 1. find top face
top = sorted(my_block.faces(), key=lambda face: Vector(* my_block.face_normal(face, unitized=True)).dot([0,0,1]))[0]
my_block.face_attribute(top, 'top', True)

'''
# 2. get a list of the top faces
top = list(my_block.faces_where({'top': True}))[0]
'''

bottom = sorted(my_block.faces(), key=lambda face: Vector(* my_block.face_normal(face, unitized=True)).dot([0,0,1]))[-1]
my_block.face_attribute(bottom, 'bottom', True)


'''
# 3. set all face colors
facecolors = {face: (0.7, 0.7, 0.7) for face in my_block.faces()}
print(facecolors[top])

# 4. set a different facecolor for the top face
facecolors[top] = (1.0, 0.8, 0.8)
print(facecolors[top])
print(facecolors)
'''

# 5. visualise individual block with top face different color
viewer = App(width=1600, height=900)
viewer.view.camera.ty = -0.8
viewer.view.camera.tx = -0.7
viewer.view.camera.distance = 10

viewer.add(my_block, facecolors={top: (255,0,0), bottom: (0,128,0)})
viewer.show()

[0, 1, 2, 3, 4, 5, 6, 7]


In [31]:
'''
# Alternative method using bottom and top face vertices

import os
import compas

from compas.datastructures  import Mesh
from compas.geometry  import Vector
from compas_view2.app import App

bottom = my_block.face_vertices(0)
top = my_block.face_vertices(1)[::-1]

for face in my_block.faces():
    print(my_block.face_vertices(face))
    if set(my_block.face_vertices(face)) == set(top):
        my_block.face_attribute(face, 'top', True)
        print('top face index', face)
    elif set(my_block.face_vertices(face)) == set(bottom):
        my_block.face_attribute(face, 'bottom', True)
        print('bottom face index', face)

# 5. visualise individual block with top face different color
viewer = App(width=1600, height=900)
viewer.view.camera.ty = -0.8
viewer.view.camera.tx = -0.7
viewer.view.camera.distance = 10

viewer.add(my_block, facecolors={top: (255,0,0), bottom: (0,128,0)})
viewer.show()
'''

"\n# Alternative method using bottom and top face vertices\n\nimport os\nimport compas\n\nfrom compas.datastructures  import Mesh\nfrom compas.geometry  import Vector\nfrom compas_view2.app import App\n\nbottom = my_block.face_vertices(0)\ntop = my_block.face_vertices(1)[::-1]\n\nfor face in my_block.faces():\n    print(my_block.face_vertices(face))\n    if set(my_block.face_vertices(face)) == set(top):\n        my_block.face_attribute(face, 'top', True)\n        print('top face index', face)\n    elif set(my_block.face_vertices(face)) == set(bottom):\n        my_block.face_attribute(face, 'bottom', True)\n        print('bottom face index', face)\n\n# 5. visualise individual block with top face different color\nviewer = App(width=1600, height=900)\nviewer.view.camera.ty = -0.8\nviewer.view.camera.tx = -0.7\nviewer.view.camera.distance = 10\n\nviewer.add(my_block, facecolors={top: (255,0,0), bottom: (0,128,0)})\nviewer.show()\n"

In [32]:
import os
import compas

from compas.datastructures  import Mesh
from compas.geometry import bestfit_frame_numpy
from compas.geometry  import Frame, Rotation, Transformation, Plane
from compas.geometry  import Box
from compas_view2.app import App

# 1. get a list of the top faces
top = list(my_block.faces_where({'top': True}))[0]

# 2. get corner vertex coordinates the top faces
corners = my_block.face_coordinates(top)
print(corners)

# 3. generate a bestfit frame of the corners
frame = Frame(*bestfit_frame_numpy(corners))


'''
################ CHECK THIS ################
EXTRA:
Since the face is 2D, the frame might be roto-reflected.
If that is the case, rotate it by 180 degrees around the frame y axis.'''

if frame.zaxis.dot([0, 1, 0]) < 0:
    angle_rad = 180
    R = Rotation.from_axis_and_angle(frame.yaxis, angle_rad, point=frame.point)
    frame.transform(R)


# 4. generate a world frame
world = Frame.worldXY()

# 5. build the frame to frame transformation
X = Transformation.from_frame_to_frame(frame, world)

# 6. perform the transformation on the block
transformed_block = my_block.transformed(X)

# 7. flip the block to make it oriented for the wirecutting
xaxis, yaxis, zaxis = [1, 0, 0], [0, 1, 0], [0, 0, 1]

X2 = Rotation.from_axis_and_angle(yaxis, 3.14159)
rotated_block = transformed_block.transformed(X2)

# 7. set new variable name for the block
final_block = rotated_block


# 9. generate the machining workspace at worldXY
machine_dim = [24.00, 12.00, 15.50]
machine_space = Box(world,machine_dim[0],machine_dim[1],machine_dim[2])


# 8. visualize the correctly oriented block within the machining workspace
viewer = App(width=1600, height=900)
viewer.view.camera.ty = -0.8
viewer.view.camera.tx = -0.7
viewer.view.camera.distance = 10

viewer.add(my_block, facecolors={top: (255,0,0)}, opacity=0.5)
# viewer.add(transformed_block, facecolors={top: (255,0,0)})
viewer.add(final_block, facecolors={top: (255,0,0)})

# viewer.add(machine_space, opacity=0.2)

viewer.show()

[[7.230564269705165, 4.848557581589693, 4.139765633420204], [5.4070400413559625, 5.407640841017567, 4.517785878631973], [4.848021697604618, 7.231081034750968, 4.139564011176455], [5.9216052876857095, 8.346651582899844, 3.4849080241529036], [7.573639817288216, 7.573972812346804, 3.222047164490389], [8.346315615879245, 5.922034529244805, 3.485091137001543]]


In [33]:
import os
import compas

from compas.geometry import Frame, Box
from compas.geometry import Scale
from compas.datastructures import Mesh
from compas.geometry import oriented_bounding_box_numpy
from compas_view2.app import App


# 1. compute the bounding box of the block mesh
#    and convert it into a box geometry object

bbox = final_block.vertices_attributes('xyz', keys=final_block.vertices())
box = oriented_bounding_box_numpy(bbox)
blank = Box.from_bounding_box(box)
blank_unsized = Box.from_bounding_box(box)
bbf = blank.frame

# 2. add padding to blank material Box object by scaling up in every direction
#    use the frame of the blank as the origin for scaling up

blank.transform(Scale.from_factors([1.10, 1.10, 1.10], frame=bbf))
thickness_blank = (blank.zsize - blank_unsized.zsize)/2


# 3. visualize the correctly oriented block within the machining workspace
viewer = App(width=1600, height=900)
viewer.view.camera.ty = -0.8
viewer.view.camera.tx = -0.7
viewer.view.camera.distance = 10

viewer.add(blank, opacity=0.5)
viewer.add(final_block)
viewer.show()

In [34]:
'''

BONUS
Pick a material, determine its standard dimensions a blank module
Compare these dimensions to the generated bounding box for the wirecut blocks
Could more than one block be cut from one unit of blank material?
What kinds of steps could be taken to minimize waste?

'''

'\n\nBONUS\nPick a material, determine its standard dimensions a blank module\nCompare these dimensions to the generated bounding box for the wirecut blocks\nCould more than one block be cut from one unit of blank material?\nWhat kinds of steps could be taken to minimize waste?\n\n'

In [80]:
import os
import compas

from compas.geometry import Frame, Box, Line
from compas.geometry import Scale
from compas.datastructures import Mesh
from compas.geometry import oriented_bounding_box_numpy
from compas_view2.app import App


for face in final_block.faces():
    if final_block.face_attribute(face,'top') == True:
        id_top = face
    if final_block.face_attribute(face,'bottom') == True:
        id_bot = face

print(id_top, id_bot)

bottom = my_block.face_vertices(0)
top = my_block.face_vertices(1)[::-1]

print(bottom[0])
print(top[0])

viewer = App(width=1600, height=900)
viewer.view.camera.ty = -0.8
viewer.view.camera.tx = -0.7
viewer.view.camera.distance = 10

for edge in final_block.edges():
    print(edge)
    if edge[0] == bottom[0] and edge[1] == top[0]:
        print('hi')
        a, b = final_block.edge_coordinates(*edge)
        print ("edge coordinates",a,b)
        line = Line(a, b)
        viewer.add(line, linecolor=(0,1.0,0))
        wire_edge = edge



# 1. find edges starting at top face and ending at bottom face
# 2. find edge that is clockwise from that edge
# 3. generate the path of the wire following those two edges
# 4. determine an approach for the inside face of the block



viewer.add(final_block)
viewer.show()

1 0
0
6
(0, 1)
(0, 5)
(0, 6)
hi
edge coordinates [1.0982117199372055, -0.6130464048908904, 2.4266404425561743] [1.7146504058062544, -0.8894468708745986, 4.549978816031613e-06]
(1, 2)
(1, 7)
(2, 3)
(2, 8)
(3, 4)
(3, 9)
(4, 5)
(4, 10)
(5, 11)
(6, 7)
(6, 11)
(7, 8)
(8, 9)
(9, 10)
(10, 11)


In [95]:
import os
import compas

from compas.geometry import Frame, Box, Line
from compas.datastructures import Mesh
from compas_view2.app import App

side_faces = final_block.edge_strip(wire_edge)

edge_pairs = []

for previous, current in zip(side_faces, side_faces[1:]):
    edge_pairs.append([previous,current])


viewer = App(width=1600, height=900)

wire_vertices = []

for pair_a, pair_b in edge_pairs:
    a, b = final_block.edge_coordinates(*pair_a)
    c, d = final_block.edge_coordinates(*pair_b)
    wire_start = Line(a, c)
    wire_end = Line(b, d)
    viewer.add(wire_start, linecolor=(0,0,1.0))
    viewer.add(wire_end, linecolor=(1.0,0,0))
    # wire_vertices.append([a,b])






for pair_a, pair_b in edge_pairs:
    print('first pair', pair_a)
    print('second pair', pair_b)

viewer.view.camera.ty = -0.8
viewer.view.camera.tx = -0.7
viewer.view.camera.distance = 10
viewer.add(final_block)
viewer.show()
    

first pair (0, 6)
second pair (1, 7)
first pair (1, 7)
second pair (2, 8)
first pair (2, 8)
second pair (3, 9)
first pair (3, 9)
second pair (4, 10)
first pair (4, 10)
second pair (5, 11)
first pair (5, 11)
second pair (0, 6)


In [85]:
import os
import compas

from compas.geometry import Frame, Box, Line, Point
from compas.datastructures import Mesh
from compas_view2.app import App

wires = []

for pair_a, pair_b in edge_pairs:

    start_a = final_block.vertex_coordinates(pair_a[0])
    print(start_a)
    pt_start_a = Point(start_a[0], start_a[1], start_a[2])
    start_b = final_block.vertex_coordinates(pair_b[0])
    pt_start_b = Point(start_b[0], start_b[1], start_b[2])

    wires.append((pt_start_a, pt_start_b))

    end_a = final_block.vertex_coordinates(pair_a[1])
    pt_end_a = Point(end_a[0], end_a[1], end_a[2])
    end_b = final_block.vertex_coordinates(pair_b[1])
    pt_end_b = Point(end_b[0], end_b[1], end_b[2])

    wires.append((pt_end_a, pt_end_b))

viewer = App(width=1600, height=900)
viewer.view.camera.ty = -0.8
viewer.view.camera.tx = -0.7
viewer.view.camera.distance = 10

viewer.add(final_block, opacity=0.2)
for a, b in wires:
    viewer.add(Line(a, b), linewidth=10, color=(1, 0, 0))

viewer.show()


TypeError: unhashable type: 'list'